In [ ]:
import pandas
import ply
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import re

data = pandas.read_csv("prelim_analysis.csv")


def tool_name(list):
    tool_list = []
    for item in list:
        item = item.replace("run.", '')
        item = item.replace(".sh", '')
        tool_list.append(item.capitalize())
    return tool_list


def length(list):
    cover = []
    for item in list:
        if "50" in item:
            cover.append(50)
        elif "75" in item:
            cover.append(75)
        elif "100" in item:
            cover.append(100)
    return cover


def coverage(list):
    coverage = []
    for item in list:
        m = re.search("cov_(\d+)_", item)
        coverage.append(m.groups()[0])
    return coverage


def dataset(list):
    dataset_list = []
    for item in list:
        if "TRA" in item:
            dataset_list.append("TRA")
        else:
            dataset_list.append("IGH")

    return dataset_list


data["Tool"] = tool_name(data["Wrapper Name"])
data["Coverage"] = coverage(data["EC Filename"])
data["Length"] = length(data["EC Filename"])
data["Base Sensitiviy"] = data["Base - TP"] / (data["Base - TP"] + data["Base - TN"])
data["Base Precision"] = data["Base - TP"] / (data["Base - TP"] + data["Base - FP"] + data["Base - FP INDEL"])
data["Base Gain"] = (data["Base - TP"] - (data["Base - FP"] + data["Base - FP INDEL"])) / \
                    (data["Base - TP"] + data["Base - FN"])

data["Dataset"] = dataset(data["EC Filename"])

z = data.groupby(["Tool", "Dataset", "Coverage"])["Base Sensitiviy"].mean()
y = data.groupby(["Tool", "Dataset", "Coverage"])["Base Precision"].mean()
x = data.groupby(["Tool", "Dataset", "Coverage"])["Base Gain"].mean()
y = y.fillna(1)
n = z.index